In [1]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor, Booster
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor, Booster
from sklearn.metrics import roc_auc_score, accuracy_score, mean_absolute_error
from sklearn.linear_model import RidgeCV
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import random, time, queue
import multiprocessing
multiprocessing.set_start_method('spawn', force= True)

from multiprocessing.managers import BaseManager

# 데이터 전처리 함수 

In [2]:
def load_data():
    train = pd.read_csv('../train.csv', encoding='euc-kr')
    test = pd.read_csv('../test.csv', encoding='euc-kr')
    train[['num', '비전기냉방설비운영','태양광보유']]
    ice={}
    hot={}
    count=0
    for i in range(0, len(train), len(train)//60):
        count +=1
        ice[count]=train.loc[i,'비전기냉방설비운영']
        hot[count]=train.loc[i,'태양광보유']
    for i in range(len(test)):
        test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
        test.loc[i, '태양광보유']=hot[test['num'][i]]
        
    return train, test

def time(x):
        return int(x[-2:])
    
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()

# 클러스터링 

In [3]:
def Cluster(data, test):
    scaler = StandardScaler()
    data_list = []
    
    for i in range(1, 61):
        d = data[data['num'] == i].copy()
        d['전력사용량(kWh)'] = scaler.fit_transform(d['전력사용량(kWh)'].values.reshape(-1,1))
        result_d = d.groupby(['weekday','time'])['전력사용량(kWh)'].mean().unstack()
        data_list.append(result_d.to_numpy())
    
    data_list = np.array(data_list)
    
    #shape를 변경하여 60으로 변경 
    data_list = data_list.reshape(60, -1)
    
    model = KMeans(init="k-means++", n_clusters=4, random_state=0)
    model.fit(data_list)
    
    cluster = model.labels_
    pd.DataFrame(cluster).value_counts()
    
    cluster_map = {i+1 : c for i, c in enumerate(cluster)}
    
    data['cluster'] = data['num'].map(cluster_map)
    test['cluster'] = test['num'].map(cluster_map)
    
    return data, test 

In [8]:
def LGBM(X, y, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True ,random_state=42)
    
    train_fold_predict = np.zeros((X.shape[0],1))
    test_predict = np.zeros((X_test.shape[0],n_folds))
    
    for fold, (train, validatin) in enumerate(kfold.split(X)):
        print(f'==========================={fold+1}==========================')
        X_train = X.iloc[train, :]
        y_train = y.iloc[train, :]
        X_valid = X.iloc[validatin, :]
        y_valid = y.iloc[validatin, :]
        #validatin = X[validatin]
        
        model=LGBMRegressor(n_estimators=10000, learning_rate=0.003 , max_bin=4096,
                            metric=['l2', 'auc'], boosting='dart',
                            max_depth=1024, objective='regression')
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
                 early_stopping_rounds=30, verbose=100)
        
        train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1)
        
        test_predict[:, fold] = model.predict(X_test) 
        print("train shape {0}".format(train_fold_predict.shape))
        print("test shape {0}".format(test_predict.shape))
        print(f'=============================================================\n\n')
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    print("test_mean shape {0}".format(test_predict_mean.shape))
    return train_fold_predict, test_predict_mean

def XGboost(X, y, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    
    train_fold_predict = np.zeros((X.shape[0],1))
    test_predict = np.zeros((X_test.shape[0], n_folds))
    
    for fold, (train, validatin) in enumerate(kfold.split(X)):
        print(f'==========================={fold+1}==========================')
        X_train = X.iloc[train, :]
        y_train = y.iloc[train, :]
        X_valid = X.iloc[validatin, :]
        y_valid = y.iloc[validatin, :]
        #validatin = X[validatin]
        
        model= XGBRegressor(n_estimators=10000, booster='gbtree', max_depth=9, 
                            eta=0.003, max_bin=1024,gamma=2,boosting='gblinear')
        
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
                 early_stopping_rounds=30, verbose=100)
        
        
        train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1) 
        
        test_predict[:, fold] = model.predict(X_test)
        print("train shape {0}".format(train_fold_predict.shape))
        print("test shape {0}".format(test_predict.shape))
        print(f'=============================================================\n\n')
        
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    print("test_mean shape {0}".format(test_predict_mean.shape))
    return train_fold_predict, test_predict_mean

def Random(X, y, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    
    train_fold_predict = np.zeros((X.shape[0],1))
    test_predict = np.zeros((X_test.shape[0], n_folds))
    
    for fold, (train, validatin) in enumerate(kfold.split(X)):
        print(f'==========================={fold+1}==========================')
        X_train = X.iloc[train, :]
        y_train = y.iloc[train, :]
        X_valid = X.iloc[validatin, :]
        y_valid = y.iloc[validatin, :]
        #validatin = X[validatin]
        
        model = RandomForestRegressor(n_estimators=1000, max_depth=64, criterion="mse"
                                    ,n_jobs=8 , verbose= True)
        model.fit(X_train, y_train)
        
        train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1) 
        
        test_predict[:, fold] = model.predict(X_test)
        print("train shape {0}".format(train_fold_predict.shape))
        print("test shape {0}".format(test_predict.shape))
        print(f'=============================================================\n\n')
        print(model.score(X_train, y_train))
        print(model.score(X_valid, y_valid))
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    print("test_mean shape {0}".format(test_predict_mean.shape))
    return train_fold_predict, test_predict_mean

def GBR(X, y, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    
    train_fold_predict = np.zeros((X.shape[0],1))
    test_predict = np.zeros((X_test.shape[0], n_folds))
    
    for fold, (train, validatin) in enumerate(kfold.split(X)):
        print(f'==========================={fold+1}==========================')
        X_train = X.iloc[train, :]
        y_train = y.iloc[train, :]
        X_valid = X.iloc[validatin, :]
        y_valid = y.iloc[validatin, :]
        #validatin = X[validatin]
        
        model = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.003, max_depth=64, 
                                          criterion="mse", loss="ls", verbose= True)
        model.fit(X_train, y_train)
        
        train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1) 
        
        test_predict[:, fold] = model.predict(X_test)
        print("train shape {0}".format(train_fold_predict.shape))
        print("test shape {0}".format(test_predict.shape))
        print(f'=============================================================\n\n')
        
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    print("test_mean shape {0}".format(test_predict_mean.shape))
    return train_fold_predict, test_predict_mean

def Extra(X, y, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    
    train_fold_predict = np.zeros((X.shape[0],1))
    test_predict = np.zeros((X_test.shape[0], n_folds))
    
    for fold, (train, validatin) in enumerate(kfold.split(X)):
        print(f'==========================={fold+1}==========================')
        X_train = X.iloc[train, :]
        y_train = y.iloc[train, :]
        X_valid = X.iloc[validatin, :]
        y_valid = y.iloc[validatin, :]
        #validatin = X[validatin]
       
        model = ExtraTreesRegressor(n_estimators=1000, random_state=3, n_jobs=8, criterion="mse", verbose=True)
        model.fit(X_train, y_train)
        
        train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1) 
        
        test_predict[:, fold] = model.predict(X_test)
        print("train shape {0}".format(train_fold_predict.shape))
        print("test shape {0}".format(test_predict.shape))
        print(f'=============================================================\n\n')
        
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    print("test_mean shape {0}".format(test_predict_mean.shape))
    return train_fold_predict, test_predict_mean

def Ada(X, y, X_test, n_folds=5):
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    
    train_fold_predict = np.zeros((X.shape[0],1))
    test_predict = np.zeros((X_test.shape[0], n_folds))
    
    for fold, (train, validatin) in enumerate(kfold.split(X)):
        print(f'==========================={fold+1}==========================')
        X_train = X.iloc[train, :]
        y_train = y.iloc[train, :]
        X_valid = X.iloc[validatin, :]
        y_valid = y.iloc[validatin, :]
        #validatin = X[validatin]
        
        model = AdaBoostRegressor(n_estimators=1000, learning_rate=0.001, random_state=3)
        model.fit(X_train, y_train)
        
        train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1) 
        
        test_predict[:, fold] = model.predict(X_test)
        print("train shape {0}".format(train_fold_predict.shape))
        print("test shape {0}".format(test_predict.shape))
        print(f'=============================================================\n\n')
        
    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)
    print("test_mean shape {0}".format(test_predict_mean.shape))
    return train_fold_predict, test_predict_mean

In [5]:
train, test = load_data()
test = test.interpolate(method='values')

train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

train_x, train_y = Cluster(train, test)

train_x = train.drop('전력사용량(kWh)',axis=1)
train_y = train[['전력사용량(kWh)']]

train_x.drop('date_time',axis=1,inplace=True)
test.drop('date_time',axis=1,inplace=True)

In [6]:
Extra_train_fold, Extra_test_predict = Extra(train_x, train_y, test)

===========================1==========================


<ipython-input-4-9027bc9a73e5>:137: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    7.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   16.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   30.3s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:   38.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0

train shape (122400, 1)
test shape (10080, 5)


===========================2==========================


<ipython-input-4-9027bc9a73e5>:137: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    6.9s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   16.4s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   30.0s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:   38.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0

train shape (122400, 1)
test shape (10080, 5)


===========================3==========================


...

train shape (122400, 1)
test shape (10080, 5)


===========================4==========================


<ipython-input-4-9027bc9a73e5>:137: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    7.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   17.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   31.1s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:   39.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0

train shape (122400, 1)
test shape (10080, 5)


===========================5==========================


<ipython-input-4-9027bc9a73e5>:137: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    6.8s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   15.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   29.9s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:   37.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0

train shape (122400, 1)
test shape (10080, 5)


test_mean shape (10080, 1)


In [9]:
Ada_train_fold, Ada_test_predict = Ada(train_x, train_y, test)

===========================1==========================


c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


train shape (122400, 1)
test shape (10080, 5)


===========================2==========================


c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


train shape (122400, 1)
test shape (10080, 5)


===========================3==========================


...

train shape (122400, 1)
test shape (10080, 5)


===========================4==========================


c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


train shape (122400, 1)
test shape (10080, 5)


===========================5==========================


c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


train shape (122400, 1)
test shape (10080, 5)


test_mean shape (10080, 1)


In [11]:
random_train_fold, random_test_predict = Random(train_x, train_y, test)

===========================1==========================


<ipython-input-8-ad923c037ddf>:79: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   10.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   23.0s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   41.9s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:   52.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.

train shape (122400, 1)
test shape (10080, 5)




[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s


0.9986169062534465


...

train shape (122400, 1)
test shape (10080, 5)




[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s


0.9986390092600759


[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.6s finished


0.989070105839394
test_mean shape (10080, 1)


In [12]:
gbr_train_fold, gbr_test_predict = GBR(train_x, train_y, test)

===========================1==========================
      Iter       Train Loss   Remaining Time 


c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


         1     4207251.0431           19.16m
         2     4182045.4028           19.06m
         3     4156990.7695           18.77m
         4     4132086.2385           18.50m
         5     4107330.9106           18.26m
         6     4082723.8918           18.03m
         7     4058264.2937           17.92m
         8     4033951.2331           17.94m
         9     4009783.8319           17.90m
        10     3985761.2177           17.91m
        20     3753310.0709           17.91m
        30     3534415.5659           17.84m
        40     3328287.0742           17.75m
        50     3134180.0767           17.49m
        60     2951393.4752           17.32m
        70     2779267.0599           17.15m
        80     2617179.1245           17.03m
        90     2464544.2209           16.87m
       100     2320811.0446           16.79m
       200     1272540.0614           15.82m
       300      697755.3251           14.93m
       400      382591.1181           14.05m
       500

c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


      Iter       Train Loss   Remaining Time 
         1     4214814.1643           19.52m
         2     4189563.2134           19.20m
         3     4164463.5409           18.86m
         4     4139514.2406           18.55m
         5     4114714.4115           18.39m
         6     4090063.1582           18.21m
         7     4065559.5906           18.08m
         8     4041202.8238           17.98m
         9     4016991.9784           17.82m
        10     3992926.1802           17.72m
        20     3760057.1701           17.52m
        30     3540769.1718           17.28m
        40     3334270.1354           17.17m
        50     3139814.2036           17.13m
        60     2956699.0178           17.04m
        70     2784263.1813           16.91m
        80     2621883.8703           16.87m
        90     2468974.5842           16.81m
       100     2324983.0276           16.70m
       200     1274827.6315           15.75m
       300      699009.6397           14.84m
       40

...

      Iter       Train Loss   Remaining Time 
         1     4251055.5117           18.62m
         2     4225587.4388           18.27m
         3     4200271.9452           18.18m
         4     4175108.1167           17.95m
         5     4150095.0447           17.94m
         6     4125231.8260           17.91m
         7     4100517.5629           17.72m
         8     4075951.3629           17.65m
         9     4051532.3390           17.66m
        10     4027259.6094           17.58m
        20     3792388.2602           17.31m
        30     3571214.6999           17.20m
        40     3362940.0683           16.97m
        50     3166812.0946           16.87m
        60     2982122.3811           16.83m
        70     2808203.8440           16.74m
        80     2644428.3040           16.65m
        90     2490204.2175           16.57m
       100     2344974.5402           16.48m
       200     1285789.3154           15.69m
       300      705020.1155           14.87m
       40

c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


      Iter       Train Loss   Remaining Time 
         1     4204339.9090           18.66m
         2     4179151.7086           18.38m
         3     4154114.4108           18.26m
         4     4129227.1113           18.22m
         5     4104488.9117           17.95m
         6     4079898.9187           17.86m
         7     4055456.2443           17.69m
         8     4031160.0059           17.65m
         9     4007009.3264           17.69m
        10     3983003.3335           17.74m
        20     3750713.0207           17.33m
        30     3531969.9699           17.07m
        40     3325984.0993           17.07m
        50     3132011.4054           17.06m
        60     2949351.2748           16.98m
        70     2777343.9546           16.91m
        80     2615368.1685           16.80m
        90     2462838.8737           16.71m
       100     2319205.1471           16.66m
       200     1271659.4675           15.75m
       300      697272.4274           15.00m
       40

c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


      Iter       Train Loss   Remaining Time 
         1     4192722.7480           18.49m
         2     4167604.1461           18.60m
         3     4142636.0296           18.28m
         4     4117817.4972           18.04m
         5     4093147.6526           17.93m
         6     4068625.6050           17.87m
         7     4044250.4690           17.74m
         8     4020021.3645           17.76m
         9     3995937.4165           17.75m
        10     3971997.7555           17.69m
        20     3740349.2923           17.45m
        30     3522210.6582           17.34m
        40     3316793.9545           17.16m
        50     3123357.2334           17.11m
        60     2941201.8175           17.10m
        70     2769669.7768           16.94m
        80     2608141.5518           16.83m
        90     2456033.7161           16.71m
       100     2312796.8689           16.58m
       200     1268145.7000           15.67m
       300      695345.7692           14.81m
       40

In [13]:
lgbm_train_fold, lgbm_test_predict = LGBM(train_x, train_y, test)

===========================1==========================
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\rlagh\appdata\local\programs\python\python38\lib\site-packages\lightgbm\callback.py:183: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	training's l2: 4.39101e+06	training's auc: 0.483792	valid_1's l2: 4.3971e+06	valid_1's auc: 0.481494
[200]	training's l2: 4.87363e+06	training's auc: 0.430543	valid_1's l2: 4.86804e+06	valid_1's auc: 0.427121
[300]	training's l2: 4.41705e+06	training's auc: 0.419533	valid_1's l2: 4.40789e+06	valid_1's auc: 0.415601
[400]	training's l2: 3.91784e+06	training's auc: 0.416653	valid_1's l2: 3.9082e+06	valid_1's auc: 0.413232
[500]	training's l2: 3.42331e+06	training's auc: 0.411674	valid_1's l2: 3.4119e+06	valid_1's auc: 0.407737
[600]	training's l2: 3.20615e+06	training's auc: 0.413114	valid_1's l2: 3.19302e+06	valid_1's auc: 0.408922
[700]	training's l2: 2.89765e+06	training's auc: 0.438483	valid_1's l2: 2.8825e+06	valid_1's auc: 0.435108
[800]	training's l2: 2.70878e+06	training's auc: 0.491988	valid_1's l2: 2.69253e+06	valid_1's auc: 0.490461
[900]	training's l2: 2.40421e+06	training's auc: 0.511173	valid_1's l2: 2.38688e+06	valid_1's auc: 0.510662
[1000]	training's l2: 2.22642e+0

[8300]	training's l2: 159876	training's auc: 0.969801	valid_1's l2: 158714	valid_1's auc: 0.972262
[8400]	training's l2: 160628	training's auc: 0.969709	valid_1's l2: 159580	valid_1's auc: 0.972425
[8500]	training's l2: 160334	training's auc: 0.969287	valid_1's l2: 159369	valid_1's auc: 0.972058
[8600]	training's l2: 158377	training's auc: 0.968289	valid_1's l2: 157479	valid_1's auc: 0.971363
[8700]	training's l2: 155934	training's auc: 0.96787	valid_1's l2: 155116	valid_1's auc: 0.971118
[8800]	training's l2: 154086	training's auc: 0.967053	valid_1's l2: 153349	valid_1's auc: 0.970383
[8900]	training's l2: 152636	training's auc: 0.967084	valid_1's l2: 151951	valid_1's auc: 0.970097
[9000]	training's l2: 148253	training's auc: 0.966328	valid_1's l2: 147612	valid_1's auc: 0.96879
[9100]	training's l2: 147290	training's auc: 0.967046	valid_1's l2: 146739	valid_1's auc: 0.969239
[9200]	training's l2: 145886	training's auc: 0.968019	valid_1's l2: 145363	valid_1's auc: 0.970015
[9300]	train

[6200]	training's l2: 221913	training's auc: 0.956527	valid_1's l2: 231201	valid_1's auc: 0.946934
[6300]	training's l2: 215333	training's auc: 0.954877	valid_1's l2: 224776	valid_1's auc: 0.946852
[6400]	training's l2: 212323	training's auc: 0.953447	valid_1's l2: 221820	valid_1's auc: 0.946526
[6500]	training's l2: 207392	training's auc: 0.953861	valid_1's l2: 216942	valid_1's auc: 0.947098
[6600]	training's l2: 201973	training's auc: 0.957541	valid_1's l2: 211571	valid_1's auc: 0.950652
[6700]	training's l2: 200283	training's auc: 0.96155	valid_1's l2: 209898	valid_1's auc: 0.953103
[6800]	training's l2: 195701	training's auc: 0.962321	valid_1's l2: 205369	valid_1's auc: 0.95633
[6900]	training's l2: 193325	training's auc: 0.96227	valid_1's l2: 203015	valid_1's auc: 0.956902
[7000]	training's l2: 190869	training's auc: 0.959627	valid_1's l2: 200599	valid_1's auc: 0.9549
[7100]	training's l2: 190169	training's auc: 0.960986	valid_1's l2: 199873	valid_1's auc: 0.956616
[7200]	training

[4100]	training's l2: 349305	training's auc: 0.951615	valid_1's l2: 339552	valid_1's auc: 0.94628
[4200]	training's l2: 343775	training's auc: 0.953507	valid_1's l2: 334186	valid_1's auc: 0.957841
[4300]	training's l2: 340096	training's auc: 0.956947	valid_1's l2: 330608	valid_1's auc: 0.961518
[4400]	training's l2: 335964	training's auc: 0.95915	valid_1's l2: 326653	valid_1's auc: 0.965072
[4500]	training's l2: 331544	training's auc: 0.962763	valid_1's l2: 322524	valid_1's auc: 0.965481
[4600]	training's l2: 330747	training's auc: 0.966411	valid_1's l2: 321761	valid_1's auc: 0.967156
[4700]	training's l2: 321156	training's auc: 0.968709	valid_1's l2: 312491	valid_1's auc: 0.968504
[4800]	training's l2: 308521	training's auc: 0.964816	valid_1's l2: 300169	valid_1's auc: 0.96401
[4900]	training's l2: 306279	training's auc: 0.968589	valid_1's l2: 297971	valid_1's auc: 0.965399
[5000]	training's l2: 290224	training's auc: 0.963631	valid_1's l2: 282361	valid_1's auc: 0.963806
[5100]	traini

[2000]	training's l2: 1.05699e+06	training's auc: 0.768295	valid_1's l2: 1.08467e+06	valid_1's auc: 0.764891
[2100]	training's l2: 950273	training's auc: 0.793333	valid_1's l2: 976459	valid_1's auc: 0.796123
[2200]	training's l2: 885130	training's auc: 0.811434	valid_1's l2: 909760	valid_1's auc: 0.815099
[2300]	training's l2: 814822	training's auc: 0.833851	valid_1's l2: 837808	valid_1's auc: 0.836384
[2400]	training's l2: 741211	training's auc: 0.824813	valid_1's l2: 762819	valid_1's auc: 0.829316
[2500]	training's l2: 709906	training's auc: 0.830283	valid_1's l2: 730738	valid_1's auc: 0.833258
[2600]	training's l2: 684919	training's auc: 0.860196	valid_1's l2: 705218	valid_1's auc: 0.86445
[2700]	training's l2: 650800	training's auc: 0.881234	valid_1's l2: 670210	valid_1's auc: 0.886551
[2800]	training's l2: 651353	training's auc: 0.896414	valid_1's l2: 670704	valid_1's auc: 0.900114
[2900]	training's l2: 622088	training's auc: 0.903318	valid_1's l2: 640600	valid_1's auc: 0.906406
[

[100]	training's l2: 4.3687e+06	training's auc: 0.46021	valid_1's l2: 4.49333e+06	valid_1's auc: 1
[200]	training's l2: 4.84887e+06	training's auc: 0.428826	valid_1's l2: 4.97159e+06	valid_1's auc: 1
[300]	training's l2: 4.3955e+06	training's auc: 0.422606	valid_1's l2: 4.49924e+06	valid_1's auc: 1
[400]	training's l2: 3.89974e+06	training's auc: 0.412342	valid_1's l2: 3.98613e+06	valid_1's auc: 1
[500]	training's l2: 3.40896e+06	training's auc: 0.432104	valid_1's l2: 3.47842e+06	valid_1's auc: 1
[600]	training's l2: 3.19302e+06	training's auc: 0.439182	valid_1's l2: 3.25729e+06	valid_1's auc: 1
[700]	training's l2: 2.88425e+06	training's auc: 0.455604	valid_1's l2: 2.9394e+06	valid_1's auc: 1
[800]	training's l2: 2.6949e+06	training's auc: 0.500357	valid_1's l2: 2.74498e+06	valid_1's auc: 1
[900]	training's l2: 2.38942e+06	training's auc: 0.526074	valid_1's l2: 2.43163e+06	valid_1's auc: 1
[1000]	training's l2: 2.21136e+06	training's auc: 0.555323	valid_1's l2: 2.24911e+06	valid_1's a

[8900]	training's l2: 149800	training's auc: 0.971555	valid_1's l2: 156597	valid_1's auc: 1
[9000]	training's l2: 145491	training's auc: 0.972392	valid_1's l2: 152321	valid_1's auc: 1
[9100]	training's l2: 144583	training's auc: 0.973663	valid_1's l2: 151403	valid_1's auc: 1
[9200]	training's l2: 142959	training's auc: 0.975019	valid_1's l2: 149833	valid_1's auc: 1
[9300]	training's l2: 141881	training's auc: 0.97556	valid_1's l2: 148787	valid_1's auc: 1
[9400]	training's l2: 140888	training's auc: 0.976002	valid_1's l2: 147781	valid_1's auc: 1
[9500]	training's l2: 139631	training's auc: 0.976314	valid_1's l2: 146521	valid_1's auc: 1
[9600]	training's l2: 137194	training's auc: 0.975918	valid_1's l2: 144111	valid_1's auc: 1
[9700]	training's l2: 135065	training's auc: 0.975638	valid_1's l2: 142021	valid_1's auc: 1
[9800]	training's l2: 133171	training's auc: 0.975585	valid_1's l2: 140165	valid_1's auc: 1
[9900]	training's l2: 130347	training's auc: 0.975981	valid_1's l2: 137390	valid_

In [14]:
xgboost_train_fold, xgboost_test_predict = XGboost(train_x, train_y, test)

===========================1==========================
[07:35:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:3096.87769	validation_1-rmse:3093.66943
[100]	validation_0-rmse:2332.73120	validation_1-rmse:2328.87036
[200]	validation_0-rmse:1780.42639	validation_1-rmse:1775.49487
[300]	validation_0-rmse:1377.41602	validation_1-rmse:1371.62769
[400]	validation_0-rmse:1073.68494	validation_1-rmse:1068.50171
[500]	validation_0-rmse:857.76416	validation_1-rmse:852.93158
[600]	validation_0-rmse:703.18396	validation_1-rmse:698.96820
[700]	validation_0-rmse:590.24585	validation_1-rmse:587.39887
[800]	validation_0-rmse:515.14795	validation_1-rmse:5

[2000]	validation_0-rmse:264.66476	validation_1-rmse:300.22024
[2100]	validation_0-rmse:256.99341	validation_1-rmse:293.88739
[2200]	validation_0-rmse:250.35397	validation_1-rmse:288.73514
[2300]	validation_0-rmse:244.33110	validation_1-rmse:284.19742
[2400]	validation_0-rmse:239.09523	validation_1-rmse:280.28980
[2500]	validation_0-rmse:234.15745	validation_1-rmse:276.76044
[2600]	validation_0-rmse:229.31438	validation_1-rmse:273.22583
[2700]	validation_0-rmse:225.05800	validation_1-rmse:270.00394
[2800]	validation_0-rmse:221.95866	validation_1-rmse:267.84775
[2900]	validation_0-rmse:218.45953	validation_1-rmse:265.35037
[3000]	validation_0-rmse:215.68903	validation_1-rmse:263.53101
[3100]	validation_0-rmse:213.40843	validation_1-rmse:262.01617
[3200]	validation_0-rmse:211.13882	validation_1-rmse:260.64529
[3300]	validation_0-rmse:208.85518	validation_1-rmse:259.16617
[3400]	validation_0-rmse:206.70419	validation_1-rmse:257.67761
[3500]	validation_0-rmse:204.34044	validation_1-rmse:25

[4100]	validation_0-rmse:197.62650	validation_1-rmse:234.89677
[4200]	validation_0-rmse:196.54610	validation_1-rmse:234.60696
[4300]	validation_0-rmse:195.32423	validation_1-rmse:234.12547
[4400]	validation_0-rmse:193.91518	validation_1-rmse:233.45418
[4500]	validation_0-rmse:192.31015	validation_1-rmse:232.65843
[4600]	validation_0-rmse:191.05737	validation_1-rmse:232.02853
[4700]	validation_0-rmse:189.38420	validation_1-rmse:231.35555
[4800]	validation_0-rmse:188.14465	validation_1-rmse:230.91498
[4900]	validation_0-rmse:187.00472	validation_1-rmse:230.57651
[5000]	validation_0-rmse:185.73381	validation_1-rmse:230.03650
[5100]	validation_0-rmse:184.97685	validation_1-rmse:229.68123
[5200]	validation_0-rmse:183.94058	validation_1-rmse:229.10196
[5300]	validation_0-rmse:182.66159	validation_1-rmse:228.39458
[5400]	validation_0-rmse:181.72488	validation_1-rmse:227.92105
[5500]	validation_0-rmse:180.81427	validation_1-rmse:227.40710
[5600]	validation_0-rmse:180.05171	validation_1-rmse:22

[8500]	validation_0-rmse:155.86435	validation_1-rmse:209.98967
[8600]	validation_0-rmse:155.34312	validation_1-rmse:209.77164
[8700]	validation_0-rmse:154.75040	validation_1-rmse:209.51447
[8800]	validation_0-rmse:154.34364	validation_1-rmse:209.37013
[8900]	validation_0-rmse:154.00981	validation_1-rmse:209.26355
[9000]	validation_0-rmse:152.93233	validation_1-rmse:208.75954
[9100]	validation_0-rmse:151.75946	validation_1-rmse:208.29131
[9200]	validation_0-rmse:151.05951	validation_1-rmse:208.08105
[9300]	validation_0-rmse:150.25746	validation_1-rmse:207.77286
[9400]	validation_0-rmse:149.78308	validation_1-rmse:207.59692
[9500]	validation_0-rmse:149.46545	validation_1-rmse:207.52986
[9600]	validation_0-rmse:149.23381	validation_1-rmse:207.49200
[9700]	validation_0-rmse:148.98628	validation_1-rmse:207.43780
[9800]	validation_0-rmse:148.65083	validation_1-rmse:207.35609
[9900]	validation_0-rmse:148.28662	validation_1-rmse:207.27257
[9999]	validation_0-rmse:147.90483	validation_1-rmse:20

In [15]:
train_pred = np.concatenate([Extra_train_fold, Ada_train_fold ,lgbm_train_fold, xgboost_train_fold, random_train_fold, gbr_train_fold], axis=1)
train_pred.shape

(122400, 6)

In [17]:
test_pred = np.concatenate([Extra_test_predict, Ada_test_predict, lgbm_test_predict, xgboost_test_predict,random_test_predict, gbr_test_predict], axis=1)
test_pred.shape

(10080, 6)

In [18]:
# 데이터 다시불러오기 
train, test = load_data()
test = test.interpolate(method='values')

train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

train_x, train_y = Cluster(train, test)

train_x = train.drop('전력사용량(kWh)',axis=1)
train_y = train['전력사용량(kWh)'].values

train_x.drop('date_time',axis=1,inplace=True)
test.drop('date_time',axis=1,inplace=True)

In [31]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
splits = folds.split(train_pred, train_y)

train_fold_predict = np.zeros((train_pred.shape[0],1))
test_predict = np.zeros((test_pred.shape[0], n_fold))
    
for fold, (train, validatin) in enumerate(splits):
    print(f'==========================={fold+1}==========================')
    X_train = train_pred[train]
    y_train = train_y[train]
    X_valid = train_pred[validatin]
    y_valid = train_y[validatin]
        
    
    '''
    model=LGBMRegressor(n_estimators=10000, learning_rate=0.003 , max_bin=4096,
                            metric=['l2', 'mse'],
                            max_depth=1024, objective='regression')
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
                 early_stopping_rounds=30, verbose=100)
    '''
    model = CatBoostRegressor(eval_metric='RMSE', iterations=10000, early_stopping_rounds=1000, task_type='GPU')
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    
    train_fold_predict[validatin, :] = model.predict(X_valid).reshape(-1,1)
    test_predict[:, fold] = model.predict(test_pred)

    print(f'=============================================================\n\n')
    
test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)

===========================1==========================
Learning rate set to 0.031785
0:	learn: 2002.5322114	test: 2011.4048263	best: 2011.4048263 (0)	total: 7ms	remaining: 1m 9s
1:	learn: 1950.2082852	test: 1959.5414163	best: 1959.5414163 (1)	total: 12.1ms	remaining: 1m
2:	learn: 1900.6637657	test: 1910.5365034	best: 1910.5365034 (2)	total: 17.6ms	remaining: 58.6s
3:	learn: 1852.1928950	test: 1862.7164477	best: 1862.7164477 (3)	total: 22.8ms	remaining: 57s
4:	learn: 1806.6375108	test: 1817.7034307	best: 1817.7034307 (4)	total: 27.9ms	remaining: 55.8s
5:	learn: 1761.7279693	test: 1773.3769949	best: 1773.3769949 (5)	total: 33.2ms	remaining: 55.2s
6:	learn: 1718.9871954	test: 1731.0394958	best: 1731.0394958 (6)	total: 38.3ms	remaining: 54.7s
7:	learn: 1678.0645434	test: 1690.5624512	best: 1690.5624512 (7)	total: 43.4ms	remaining: 54.3s
8:	learn: 1639.2371125	test: 1652.1528993	best: 1652.1528993 (8)	total: 48.7ms	remaining: 54.1s
9:	learn: 1600.2234615	test: 1613.4826961	best: 1613.482696

101:	learn: 697.4358864	test: 707.8250218	best: 707.8250218 (101)	total: 539ms	remaining: 52.3s
102:	learn: 696.8544976	test: 707.1360335	best: 707.1360335 (102)	total: 545ms	remaining: 52.4s
103:	learn: 696.2618394	test: 706.5370210	best: 706.5370210 (103)	total: 550ms	remaining: 52.4s
104:	learn: 695.9144114	test: 706.1349979	best: 706.1349979 (104)	total: 556ms	remaining: 52.4s
105:	learn: 695.6076424	test: 705.7772575	best: 705.7772575 (105)	total: 561ms	remaining: 52.4s
106:	learn: 682.9716843	test: 693.2886863	best: 693.2886863 (106)	total: 566ms	remaining: 52.3s
107:	learn: 682.8010284	test: 693.0915717	best: 693.0915717 (107)	total: 571ms	remaining: 52.3s
108:	learn: 682.3086990	test: 692.5873513	best: 692.5873513 (108)	total: 576ms	remaining: 52.3s
109:	learn: 681.8594186	test: 692.0933406	best: 692.0933406 (109)	total: 582ms	remaining: 52.3s
110:	learn: 671.9370963	test: 682.2208716	best: 682.2208716 (110)	total: 587ms	remaining: 52.3s
111:	learn: 671.7827847	test: 682.039724

203:	learn: 466.0269004	test: 477.2975559	best: 477.2975559 (203)	total: 1.07s	remaining: 51.5s
204:	learn: 465.4344067	test: 476.6760948	best: 476.6760948 (204)	total: 1.08s	remaining: 51.5s
205:	learn: 463.3774052	test: 474.5674494	best: 474.5674494 (205)	total: 1.08s	remaining: 51.5s
206:	learn: 461.5383428	test: 472.7298224	best: 472.7298224 (206)	total: 1.09s	remaining: 51.5s
207:	learn: 459.8080873	test: 470.9827745	best: 470.9827745 (207)	total: 1.09s	remaining: 51.5s
208:	learn: 458.8876121	test: 470.0125655	best: 470.0125655 (208)	total: 1.1s	remaining: 51.5s
209:	learn: 458.3784776	test: 469.4445724	best: 469.4445724 (209)	total: 1.1s	remaining: 51.5s
210:	learn: 456.8521793	test: 467.9188355	best: 467.9188355 (210)	total: 1.11s	remaining: 51.5s
211:	learn: 455.2607010	test: 466.2756233	best: 466.2756233 (211)	total: 1.11s	remaining: 51.5s
212:	learn: 453.8069959	test: 464.7821670	best: 464.7821670 (212)	total: 1.12s	remaining: 51.5s
213:	learn: 452.3262169	test: 463.2630840	

302:	learn: 424.0046368	test: 433.8141284	best: 433.8141284 (302)	total: 1.61s	remaining: 51.5s
303:	learn: 423.9678617	test: 433.7967236	best: 433.7967236 (303)	total: 1.61s	remaining: 51.5s
304:	learn: 423.9396923	test: 433.7621287	best: 433.7621287 (304)	total: 1.62s	remaining: 51.5s
305:	learn: 423.9251136	test: 433.7429376	best: 433.7429376 (305)	total: 1.63s	remaining: 51.5s
306:	learn: 423.9201553	test: 433.7367413	best: 433.7367413 (306)	total: 1.63s	remaining: 51.5s
307:	learn: 423.8956586	test: 433.7257710	best: 433.7257710 (307)	total: 1.64s	remaining: 51.5s
308:	learn: 423.8915141	test: 433.7201290	best: 433.7201290 (308)	total: 1.64s	remaining: 51.5s
309:	learn: 423.8688661	test: 433.7106050	best: 433.7106050 (309)	total: 1.65s	remaining: 51.5s
310:	learn: 423.8582572	test: 433.6959207	best: 433.6959207 (310)	total: 1.65s	remaining: 51.5s
311:	learn: 423.8446380	test: 433.6774742	best: 433.6774742 (311)	total: 1.66s	remaining: 51.5s
312:	learn: 423.8350154	test: 433.664573

401:	learn: 420.1468277	test: 430.0285687	best: 430.0285687 (401)	total: 2.14s	remaining: 51.1s
402:	learn: 420.1468029	test: 430.0280094	best: 430.0280094 (402)	total: 2.15s	remaining: 51.1s
403:	learn: 420.1465540	test: 430.0267935	best: 430.0267935 (403)	total: 2.15s	remaining: 51.1s
404:	learn: 420.1462553	test: 430.0256019	best: 430.0256019 (404)	total: 2.15s	remaining: 51.1s
405:	learn: 420.1460313	test: 430.0244103	best: 430.0244103 (405)	total: 2.16s	remaining: 51.1s
406:	learn: 420.1458072	test: 430.0233160	best: 430.0233160 (406)	total: 2.17s	remaining: 51.1s
407:	learn: 420.1457326	test: 430.0229755	best: 430.0229755 (407)	total: 2.17s	remaining: 51.1s
408:	learn: 420.1456828	test: 430.0226107	best: 430.0226107 (408)	total: 2.18s	remaining: 51s
409:	learn: 420.1456330	test: 430.0222946	best: 430.0222946 (409)	total: 2.18s	remaining: 51s
410:	learn: 420.1456081	test: 430.0220028	best: 430.0220028 (410)	total: 2.19s	remaining: 51s
411:	learn: 420.1455832	test: 430.0216623	best

...

3076:	learn: 307.7544391	test: 320.9072759	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3077:	learn: 307.7544391	test: 320.9073085	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3078:	learn: 307.7544221	test: 320.9072922	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3079:	learn: 307.7541502	test: 320.9073248	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3080:	learn: 307.7541672	test: 320.9073248	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3081:	learn: 307.7541502	test: 320.9074063	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3082:	learn: 307.7541672	test: 320.9073900	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3083:	learn: 307.7541672	test: 320.9074063	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3084:	learn: 307.7541502	test: 320.9074226	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3085:	learn: 307.7541502	test: 320.9074226	best: 320.8430887 (2451)	total: 17.3s	remaining: 38.9s
3086:	learn: 307.754

3167:	learn: 307.7471673	test: 320.9109257	best: 320.8430887 (2451)	total: 17.8s	remaining: 38.4s
3168:	learn: 307.7471333	test: 320.9109094	best: 320.8430887 (2451)	total: 17.8s	remaining: 38.4s
3169:	learn: 307.7470993	test: 320.9109094	best: 320.8430887 (2451)	total: 17.8s	remaining: 38.4s
3170:	learn: 307.7471333	test: 320.9108768	best: 320.8430887 (2451)	total: 17.8s	remaining: 38.4s
3171:	learn: 307.7471163	test: 320.9109094	best: 320.8430887 (2451)	total: 17.8s	remaining: 38.4s
3172:	learn: 307.7470993	test: 320.9109420	best: 320.8430887 (2451)	total: 17.9s	remaining: 38.4s
3173:	learn: 307.7470483	test: 320.9109257	best: 320.8430887 (2451)	total: 17.9s	remaining: 38.4s
3174:	learn: 307.7470823	test: 320.9109257	best: 320.8430887 (2451)	total: 17.9s	remaining: 38.4s
3175:	learn: 307.7470483	test: 320.9109420	best: 320.8430887 (2451)	total: 17.9s	remaining: 38.4s
3176:	learn: 307.7470653	test: 320.9109257	best: 320.8430887 (2451)	total: 17.9s	remaining: 38.4s
3177:	learn: 307.747

3258:	learn: 307.7411356	test: 320.9107302	best: 320.8430887 (2451)	total: 18.4s	remaining: 38s
3259:	learn: 307.7410846	test: 320.9107627	best: 320.8430887 (2451)	total: 18.4s	remaining: 38s
3260:	learn: 307.7411016	test: 320.9107953	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3261:	learn: 307.7411016	test: 320.9108116	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3262:	learn: 307.7410846	test: 320.9108442	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3263:	learn: 307.7411016	test: 320.9108442	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3264:	learn: 307.7410846	test: 320.9108605	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3265:	learn: 307.7411016	test: 320.9108768	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3266:	learn: 307.7411016	test: 320.9109257	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3267:	learn: 307.7410676	test: 320.9109420	best: 320.8430887 (2451)	total: 18.4s	remaining: 37.9s
3268:	learn: 307.7410846

3351:	learn: 307.7381622	test: 320.9125876	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3352:	learn: 307.7381452	test: 320.9126039	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3353:	learn: 307.7381622	test: 320.9125876	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3354:	learn: 307.7381792	test: 320.9126039	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3355:	learn: 307.7381962	test: 320.9126039	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3356:	learn: 307.7379413	test: 320.9126691	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3357:	learn: 307.7374995	test: 320.9127668	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3358:	learn: 307.7374826	test: 320.9127831	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3359:	learn: 307.7374995	test: 320.9128157	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3360:	learn: 307.7375165	test: 320.9128483	best: 320.8430887 (2451)	total: 18.9s	remaining: 37.4s
3361:	learn: 307.737

3439:	learn: 307.7230229	test: 320.9150968	best: 320.8430887 (2451)	total: 19.4s	remaining: 37s
3440:	learn: 307.7224282	test: 320.9146080	best: 320.8430887 (2451)	total: 19.4s	remaining: 37s
3441:	learn: 307.7223942	test: 320.9146569	best: 320.8430887 (2451)	total: 19.4s	remaining: 37s
3442:	learn: 307.7222413	test: 320.9156182	best: 320.8430887 (2451)	total: 19.4s	remaining: 37s
3443:	learn: 307.7210349	test: 320.9154227	best: 320.8430887 (2451)	total: 19.5s	remaining: 37s
3444:	learn: 307.7201853	test: 320.9142495	best: 320.8430887 (2451)	total: 19.5s	remaining: 37s
3445:	learn: 307.7181802	test: 320.9123921	best: 320.8430887 (2451)	total: 19.5s	remaining: 37s
3446:	learn: 307.7181122	test: 320.9123921	best: 320.8430887 (2451)	total: 19.5s	remaining: 37s
3447:	learn: 307.7180783	test: 320.9124410	best: 320.8430887 (2451)	total: 19.5s	remaining: 37s
3448:	learn: 307.7180783	test: 320.9124410	best: 320.8430887 (2451)	total: 19.5s	remaining: 37s
3449:	learn: 307.7180103	test: 320.91244

In [32]:
submission = pd.read_csv("../sample_submission.csv")
submission['answer'] = test_predict_mean
submission

,num_date_time,answer
0,1 2020-08-25 00,8627.459552
1,1 2020-08-25 01,8627.459552
2,1 2020-08-25 02,8627.459552
3,1 2020-08-25 03,8627.459552
4,1 2020-08-25 04,8627.459552
...,...,...
10075,60 2020-08-31 19,3740.696684
10076,60 2020-08-31 20,3464.533250
10077,60 2020-08-31 21,3496.094945
10078,60 2020-08-31 22,3299.649898


In [33]:
submission.to_csv("result_add_Cat_stacking_ensemble_submit.csv", index=False)